In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

import os
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"

import subprocess
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"], env=env)

import threading
thread = threading.Thread(target=run_ollama_serve)
thread.start()

import time
time.sleep(5)

In [ ]:
!ollama pull gpt-oss:20b
!curl http://localhost:11434/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'

#curl http://n2.ckey.vn:2421/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

# TODO: Chạy lệnh dưới để tunnel ra bên ngoài CHÚ Ý: NHẤN PHẢI RỒI CHỌN COPY TRONG MENU, NẾU BẤM CTRL + C SẼ TẮT KẾT NỐI
# ssh -p 443 -R0:localhost:11434 qr@a.pinggy.io

In [ ]:
# Thay "http://localhost:11434" bằng link sinh ra từ pingy.io, ví dụ nếu thấy "http://bsqlv-34-125-123-92.a.free.pinggy.link"
!curl http://tytji-34-124-205-38.a.free.pinggy.link/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Tell me a joke", "stream": false}'

In [ ]:
# Test gọi Ollama cục bộ
%pip install ollama
import ollama
response = ollama.chat(model='gpt-oss:20b', messages=[
  {
    'role': 'user',
    'content': 'Tell me a joke',
  },
])
count = response["eval_count"]
time = response["eval_duration"] / 1000000000
avg_tokens = round(count / time, 1)
print(f'{avg_tokens} tokens/s')
print(response['message']['content'])

In [ ]:
# NẾU KHÔNG MUỐN DÙNG PINGGY MÀ DÙNG NGROK THÌ SAO?
#TODO: Vào ngrok.com đăng kí một account để lấy API-KEY
%pip install pyngrok
from pyngrok import ngrok

from google.colab import userdata
ngrok_api_key = userdata.get('ngrok_API_KEY_HERE')
ngrok.set_auth_token(ngrok_api_key)

# ngrok có giới hạn 1 GB/tháng
# zrok giới hạn là 5 GB/ngày
# https://github.com/int11/Kaggle_remote_zrok
# https://www.kaggle.com/code/kayak0/kaggle-zrok


port = 11434 # Mặc định Ollama chạy ở port này
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel {public_url} -> http://127.0.0.1:{port}")

In [ ]:
!curl https://a7f3566c1126.ngrok-free.app/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
!pip install fastapi nest-asyncio pyngrok uvicorn

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get('/ask')
async def ask(question: str):
    return {
        'question': question,
        'answer': "I don't know"
    }

import threading
import uvicorn

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("Server started on http://0.0.0.0:8000")


In [ ]:
!curl localhost:8000

In [ ]:
!curl localhost:8000/ask?question=Tell%20me%20a%20joke

In [ ]:
# Kill the ngrok process, closing all active tunnels
ngrok.kill()